#### Prepared for Gabor's Data Analysis

### Data Analysis for Business, Economics, and Policy
by Gabor Bekes and  Gabor Kezdi
 
Cambridge University Press 2021

**[gabors-data-analysis.com ](https://gabors-data-analysis.com/)**

 License: Free to share, modify and use for educational purposes. 
 Not to be used for commercial purposes.

### Chapter 16
**CH16A Predicting apartment prices with random forest**

using the airbnb dataset

version 1.0 2021-05-05

In [22]:
import os
import sys
import warnings
from datetime import datetime as dt

import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
from IPython.core.display import HTML
from plotnine import *
from stargazer.stargazer import Stargazer
import pyfixest as pf

warnings.filterwarnings("ignore")


In [23]:
# Current script folder
current_path = os.getcwd()
dirname = current_path.split("da_case_studies")[0]

# location folders
data_in = dirname + "da_data_repo/worldbank-immunization/clean/"
data_out = dirname + "da_case_studies/ch23-immunization-life/"
output = dirname + "da_case_studies/ch23-immunization-life/output/"

func = dirname + "da_case_studies/ch00-tech-prep/"
sys.path.append(func)


In [24]:
# Import the prewritten helper functions
from py_helper_functions import *


In [25]:
# Import data

data = pd.read_csv("C:/Users/alexa/Documents/da_data_repo/worldbank-immunization/worldbank-immunization-continents.csv")

In [ ]:
p1 = (
    ggplot(data, aes(x="year", y="imm_SAS"))
    + geom_line(color=color[0], size=1)
    + geom_line(aes(x="year", y="imm_SSF"), color=color[1], size=1)
    + annotate(
        geom="text",
        x=data.loc[12, "year"] - 2,
        y=data.loc[12, "imm_SAS"],
        label="South Asia",
        size=10,
    )
    + annotate(
        geom="text",
        x=data.loc[16, "year"],
        y=data.loc[16, "imm_SSF"] - 2,
        label="sub-Saharan Africa",
        size=10,
    )
    + labs(y="Immunization rate (percent)", x="Date (year)")
    + scale_y_continuous(expand=(0, 0), breaks=seq(50, 100, 10), limits=(50, 100))
    + scale_x_continuous(
        expand=(0, 0), breaks=seq(1998, 2018, 5), limits=(1998, 2018)
    )
    + theme_bw()
)

for col in data.columns[1:6]:
    p1 += geom_line(aes(x="year", y=col), color="grey", size=0.7)

p1


In [ ]:
for col in data.columns[8:]:
    data[col] = data[col] / 10


In [ ]:
p2 = (
    ggplot(data, aes(x="year", y="surv_SAS"))
    + geom_line(color=color[0], size=1)
    + geom_line(aes(x="year", y="surv_SSF"), color=color[1], size=1)
    + annotate(
        geom="text",
        x=data.loc[12, "year"] - 4,
        y=data.loc[12, "surv_SAS"],
        label="South Asia",
        size=10,
    )
    + annotate(
        geom="text",
        x=data.loc[16, "year"],
        y=data.loc[16, "surv_SSF"] - 2,
        label="sub-Saharan Africa",
        size=10,
    )
    + labs(y="Child survival rate (percent)", x="Date (year)")
    + scale_y_continuous(expand=(0, 0), breaks=seq(80, 100, 5), limits=(80, 100))
    + scale_x_continuous(
        expand=(0, 0), breaks=range(1998, 2019, 5), limits=(1998, 2018)
    )
    + theme_bw()
)

for col in data.columns[8:13]:
    p2 += geom_line(aes(x="year", y=col), color="grey", size=0.7)

p2


# Regressions on countries


In [26]:
data_panel = pd.read_csv("C:/Users/alexa/Documents/da_data_repo/worldbank-immunization/worldbank-immunization-panel.csv")
data_panel.describe()


,year,pop,mort,surv,imm,gdppc,lngdppc,hexp
count,3807.000000,3807.000000,3807.000000,3807.000000,3807.000000,3642.000000,3642.000000,3165.000000
mean,2007.526924,35.067473,42.982847,95.701715,85.298135,16337.051802,9.042446,6.274914
std,5.759522,132.455594,45.517445,4.551744,15.738758,19070.325200,1.234401,2.878283
min,1998.000000,0.009332,2.100000,75.720000,8.000000,275.518093,5.618653,1.024978
25%,2003.000000,1.756771,9.300000,93.425000,79.000000,3114.288251,8.043756,4.321824
50%,2008.000000,7.395599,23.300000,97.670000,92.000000,9386.069679,9.146982,5.766445
75%,2013.000000,23.576856,65.750000,99.070000,96.000000,22449.101714,10.019005,7.916312
max,2017.000000,1386.395000,242.800000,99.790000,99.000000,124024.568165,11.728235,27.417822


In [27]:
data_panel = data_panel.dropna(subset=["imm", "gdppc"])


In [28]:
data_panel["balanced"] = data_panel["c"].isin(
    data_panel.groupby("c")
    .agg(
        min_year=("year", min), max_year=("year", max), n_unique_years=("year", "count")
    )
    .query("(min_year == 1998)&(max_year == 2017)&(n_unique_years==20)")
    .index
)


In [29]:
data_balanced = data_panel.query("balanced == True")


In [30]:
countries_grouped = data_balanced.groupby("c")

data_balanced["lnpop"] = countries_grouped["pop"].transform(np.log)
data_balanced["d_surv"] = countries_grouped["surv"].transform("diff")
data_balanced["d_imm"] = countries_grouped["imm"].transform("diff")
data_balanced["d2_imm"] = countries_grouped["d_imm"].transform("diff")
data_balanced["d_lngdppc"] = countries_grouped["lngdppc"].transform("diff")
data_balanced["d_lnpop"] = countries_grouped["lnpop"].transform("diff")
data_balanced["avgpop"] = countries_grouped["pop"].transform("mean")

data_balanced = data_balanced.sort_values(by=["c", "year"])


## Fixed Effect

In [31]:
import pyfixest as pf

In [32]:
data_balanced = data_balanced.assign(
    Year=lambda x: x["year"].astype("category")
).set_index(["c", "year"])


In [33]:
fe_lm = pf.feols(
    fml = "surv ~ imm | Year + c", 
    data = data_balanced.reset_index(), 
    weights = "avgpop", 
    vcov = {"CRV1": "c"}
)

pf.etable(fe_lm)

GT(_tbl_data=                                   (1)     rowtype
0           imm  0.077*** <br> (0.010)        coef
1          Year                      x          fe
2             c                      x          fe
3  Observations                   3440  modelstats
4     S.E. type                  by: c  modelstats
5            R2                      -  modelstats, _body=<great_tables._gt_data.Body object at 0x0000014932130DC0>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000014932130460>, _spanners=Spanners([SpannerInfo(spanner_id='surv', spanner_level=1, spanner_label='surv', spanner_units=None, spanner_pattern=None, vars=['(1)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x0000014932130940>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'

In [34]:
# r2 measures currently not supported via pyfixest with WLS
# fe_lm._r2_within

In [35]:
fe_lm2 = pf.feols(
    fml = "surv ~ imm + lngdppc + lnpop | Year + c", 
    data = data_balanced.reset_index(), 
    weights = "avgpop", 
    vcov = {"CRV1": "c"}
)

pf.etable([fe_lm, fe_lm2])

GT(_tbl_data=                                   (1)                     (2)     rowtype
0           imm  0.077*** <br> (0.010)   0.038*** <br> (0.011)        coef
1       lngdppc                          1.593*** <br> (0.398)        coef
2         lnpop                         12.049*** <br> (1.643)        coef
3          Year                      x                       x          fe
4             c                      x                       x          fe
5  Observations                   3440                    3440  modelstats
6     S.E. type                  by: c                   by: c  modelstats
7            R2                      -                       -  modelstats, _body=<great_tables._gt_data.Body object at 0x0000014932133820>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000149320A3400>, _spanners=Spanners([SpannerInfo(spanner_id='surv', spanner_level=1, spanner_label='surv', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000149320A3DC0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_bottom_width=O

In [36]:
# within R2
fe_lm2._r2_within


nan

In [37]:
# no weights, not in book
fe_lm2_nowts = pf.feols(
    fml = "surv ~ imm + lngdppc + lnpop | Year + c", 
    data = data_balanced.reset_index(), 
    vcov = {"CRV1": "c"}
)

pf.etable([fe_lm, fe_lm2, fe_lm2_nowts])

GT(_tbl_data=                                   (1)                     (2)                    (3)     rowtype
0           imm  0.077*** <br> (0.010)   0.038*** <br> (0.011)  0.038*** <br> (0.009)        coef
1       lngdppc                          1.593*** <br> (0.398)  2.579*** <br> (0.445)        coef
2         lnpop                         12.049*** <br> (1.643)  7.717*** <br> (1.474)        coef
3          Year                      x                       x                      x          fe
4             c                      x                       x                      x          fe
5  Observations                   3440                    3440                   3440  modelstats
6     S.E. type                  by: c                   by: c                  by: c  modelstats
7            R2                      -                       -                  0.955  modelstats, _body=<great_tables._gt_data.Body object at 0x000001493097BA00>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='(3)', type=<ColInfoTypeEnum.default: 1>, column_label='(3)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x0000014930979300>, _spanners=Spanners([SpannerInfo(spanner_id='surv', spanner_level=1, spanner_label='surv', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)', '(3)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x000001493097A890>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value

In [38]:
# large difference in R2
print(fe_lm2_nowts._r2_within)
print(fe_lm2._r2_within)


0.35603582860970395
nan
0.35603582860970395
nan


In [39]:
# CLUSTER SE VS BIASED SE

fe_lm3 = pf.feols(
    fml = "surv ~ imm + lngdppc + lnpop | Year + c", 
    data = data_balanced.reset_index(), 
    weights = "avgpop", 
    vcov = "hetero"
)

In [40]:
pf.etable([fe_lm2, fe_lm3])

GT(_tbl_data=                                    (1)                     (2)     rowtype
0           imm   0.038*** <br> (0.011)   0.038*** <br> (0.004)        coef
1       lngdppc   1.593*** <br> (0.398)   1.593*** <br> (0.119)        coef
2         lnpop  12.049*** <br> (1.643)  12.049*** <br> (0.501)        coef
3          Year                       x                       x          fe
4             c                       x                       x          fe
5  Observations                    3440                    3440  modelstats
6     S.E. type                   by: c                  hetero  modelstats
7            R2                       -                       -  modelstats, _body=<great_tables._gt_data.Body object at 0x00000149307B1E70>, _boxhead=Boxhead([ColInfo(var='', type=<ColInfoTypeEnum.stub: 2>, column_label='', column_align='left', column_width=None), ColInfo(var='(1)', type=<ColInfoTypeEnum.default: 1>, column_label='(1)', column_align='center', column_width=None), ColInfo(var='(2)', type=<ColInfoTypeEnum.default: 1>, column_label='(2)', column_align='center', column_width=None), ColInfo(var='rowtype', type=<ColInfoTypeEnum.row_group: 3>, column_label='rowtype', column_align='center', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x00000149307B1A80>, _spanners=Spanners([SpannerInfo(spanner_id='surv', spanner_level=1, spanner_label='surv', spanner_units=None, spanner_pattern=None, vars=['(1)', '(2)'], built=None)]), _heading=Heading(title=None, subtitle=None, preheader=None), _stubhead=None, _source_notes=['Significance levels: * p < 0.05, ** p < 0.01, *** p < 0.001. Format of coefficient cell:\nCoefficient \n (Std. Error)'], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x00000149307B2EF0>, _formats=[], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='hidden'), table_border_botto